In [5]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import unidecode
import string
tqdm_notebook.pandas()


load_dotenv('../secrets/.env', override=True)

scope = ["playlist-modify-private","playlist-read-private","user-library-modify","user-library-read","user-top-read"]

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
spotify.auth_manager.get_access_token(as_dict=False, check_cache=False)

my_id = spotify.current_user()['id']

%run useful_functions.ipynb
%run spotify_scraping.ipynb
%run spotify_scraping_playlists.ipynb

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

#set(master_df['Show Name'])

In [3]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Indie Show with Jack Saunders"
description = "Songs from the Indie Show with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1209/1209 [00:08<00:00, 147.47it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,5nHabL7bBWUfPArFMEbHXo,2KZedcpHb83f2AXlBDprAV,0,2023-10-22T22:22:56Z
0,2esepe5BORsvUyEXqcIQd4,2KZedcpHb83f2AXlBDprAV,1,2023-10-20T21:50:26Z
0,4OW6hRLu8typDbJ4dT3LnY,2KZedcpHb83f2AXlBDprAV,2,2023-11-05T22:48:01Z
0,1vJmd8YIIAz6XQ9GHlUr5p,2KZedcpHb83f2AXlBDprAV,3,2023-10-14T14:15:52Z
0,0UNDrAptMY5glGrcdr93Kx,2KZedcpHb83f2AXlBDprAV,4,2023-11-19T22:00:48Z
...,...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,2KZedcpHb83f2AXlBDprAV,1204,2022-12-21T22:00:32Z
0,2JjDl1YbzaBKlhSrUXZqNM,2KZedcpHb83f2AXlBDprAV,1205,2022-12-21T22:00:32Z
0,0AIlxnJ1kFpu6wp89rjjUv,2KZedcpHb83f2AXlBDprAV,1206,2022-12-21T22:00:32Z
0,7BZWTu4Pv77TxGmmsQaSXN,2KZedcpHb83f2AXlBDprAV,1207,2022-12-21T22:00:32Z


In [4]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Sounds with Clara Amfo")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Sounds with Clara Amfo"
description = "Songs from Future Sounds with Clara Amfo in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1542/1542 [00:14<00:00, 107.21it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,4gioGwFuDOfkNuDPflzjad,3VTYIgusziIIdXiWB5hB2f,0,2023-11-19T22:01:26Z
0,0UNDrAptMY5glGrcdr93Kx,3VTYIgusziIIdXiWB5hB2f,1,2023-10-20T21:50:55Z
0,7iUvtFAYg293pzSfKNBGlG,3VTYIgusziIIdXiWB5hB2f,2,2023-09-26T10:21:13Z
0,3necmcKMz1nk4r5pPbMufR,3VTYIgusziIIdXiWB5hB2f,3,2023-09-26T10:21:13Z
0,13swfFYhwoWSLEapqzKuOu,3VTYIgusziIIdXiWB5hB2f,4,2023-10-14T14:16:29Z
...,...,...,...,...
0,74RWuKK8uKMbxj4KnJpEgG,3VTYIgusziIIdXiWB5hB2f,1537,2022-12-21T22:03:34Z
0,6UoIrO15w3w62Jj8kGCnR7,3VTYIgusziIIdXiWB5hB2f,1538,2022-12-21T22:03:34Z
0,5WnvDdcyrFjbIqRxiqlet1,3VTYIgusziIIdXiWB5hB2f,1539,2022-12-21T22:03:34Z
0,6JkuJREwd6rd7t1YN069LK,3VTYIgusziIIdXiWB5hB2f,1540,2022-12-21T22:03:34Z


In [6]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Artists with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Artists with Jack Saunders"
description = "Songs from Future Artists with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 2349/2349 [00:18<00:00, 128.63it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,5sADa6C5sKDyI6PX76Bd7L,3Wo1IZcPFAfrQJzBpW8QbG,0,2023-10-14T14:17:30Z
0,2W9V8KKDPbvm5Lu7lVjUDm,3Wo1IZcPFAfrQJzBpW8QbG,1,2023-10-28T23:02:52Z
0,1eUW1kuvyCq6oFUXNLIumU,3Wo1IZcPFAfrQJzBpW8QbG,2,2023-10-14T14:17:30Z
0,3Zu4WIeXp4NbQZbcqv1MOr,3Wo1IZcPFAfrQJzBpW8QbG,3,2023-11-19T22:03:00Z
0,1McBsb9H3rFIl5cENthWd6,3Wo1IZcPFAfrQJzBpW8QbG,4,2023-10-14T14:17:30Z
...,...,...,...,...
0,4IqwX48XrtkoqvZw4hxEfa,3Wo1IZcPFAfrQJzBpW8QbG,2344,2022-12-21T22:04:15Z
0,2vT3lchY4bU1BwM69CtDyj,3Wo1IZcPFAfrQJzBpW8QbG,2345,2022-12-21T22:04:15Z
0,2KUl5KfFf1eISxtQ3xxxlQ,3Wo1IZcPFAfrQJzBpW8QbG,2346,2023-09-30T13:34:49Z
0,5cRhrmpBwkZjJ7eFMJKbVM,3Wo1IZcPFAfrQJzBpW8QbG,2347,2022-12-21T22:04:15Z


In [7]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Alternative")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Alternative"
description = "Songs from Future Alternative Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1107/1107 [00:02<00:00, 435.11it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,7LJETXmpThAx6IvUzczYNR,0U2xXmHS8QvPrX2WqGT1UH,0,2023-11-19T22:03:54Z
0,417LUyA3uRgrXVSFSKcQsr,0U2xXmHS8QvPrX2WqGT1UH,1,2023-10-28T23:03:36Z
0,06JGsU2snVzhic3RegsYXx,0U2xXmHS8QvPrX2WqGT1UH,2,2023-11-19T22:03:54Z
0,5wodknNeTlUk8E0EzKPQ3r,0U2xXmHS8QvPrX2WqGT1UH,3,2023-11-19T22:03:54Z
0,2VMU52Ncejl1gqYXBw8YUl,0U2xXmHS8QvPrX2WqGT1UH,4,2023-11-19T22:03:54Z
...,...,...,...,...
0,5QV6DCj4kvteYBDD9Vt1jY,0U2xXmHS8QvPrX2WqGT1UH,1102,2022-12-21T22:05:08Z
0,6L4L6bQPHojlmwBZQLQZjc,0U2xXmHS8QvPrX2WqGT1UH,1103,2022-12-21T22:05:08Z
0,5P0iRet3L91fC6SipKKsPl,0U2xXmHS8QvPrX2WqGT1UH,1104,2022-12-21T22:05:08Z
0,4ZG0EaIiFpFOaJKmT1ZjNO,0U2xXmHS8QvPrX2WqGT1UH,1105,2022-12-21T22:05:08Z


In [8]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'New Music Fix')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'New Music Fix'
description = "Songs from New Music Fix in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 4259/4259 [07:53<00:00,  8.99it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,7LHCsxtZf23uTUZItk4iXd,0Cwr3XcLdGZ5ZqDtI7lL11,0,2023-11-19T22:04:54Z
0,6hp729YavhhHeH62B2QH6J,0Cwr3XcLdGZ5ZqDtI7lL11,1,2023-11-19T22:04:54Z
0,4UQx1lUIMGpQStpkb2PXyr,0Cwr3XcLdGZ5ZqDtI7lL11,2,2023-10-28T23:06:20Z
0,4uHSypi6xnu8YlmLjC3tns,0Cwr3XcLdGZ5ZqDtI7lL11,3,2023-11-19T22:04:54Z
0,3neH0qCEd7nNYPOJCoB7xr,0Cwr3XcLdGZ5ZqDtI7lL11,4,2023-11-19T22:04:54Z
...,...,...,...,...
0,3Cy088VMsvC4xsIQd30UnX,0Cwr3XcLdGZ5ZqDtI7lL11,4254,2023-10-01T21:54:50Z
0,0RTuA2AWVjFw0MURes4kse,0Cwr3XcLdGZ5ZqDtI7lL11,4255,2022-12-21T22:07:39Z
0,1MCrtBVXxy2p48uWwnBwEz,0Cwr3XcLdGZ5ZqDtI7lL11,4256,2022-12-21T22:07:39Z
0,7xfwMBuatcfbzTSWRfo49f,0Cwr3XcLdGZ5ZqDtI7lL11,4257,2022-12-21T22:07:39Z


In [9]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Chillest Show")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Chillest Show"
description = "Songs from the Chillest Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1726/1726 [00:05<00:00, 308.68it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,5PJH1U5Iie893v48Fl9yaC,1p5mKMwcQVCeQNVkzpUZqJ,0,2023-11-19T22:13:52Z
0,0Z20yeDEotfT3LX8e6kSpk,1p5mKMwcQVCeQNVkzpUZqJ,1,2023-10-01T22:22:38Z
0,1KyLAl3R61UiXK9o14SUJm,1p5mKMwcQVCeQNVkzpUZqJ,2,2023-11-19T22:13:52Z
0,2ffYrXblmqpIZGdQ4BsV3g,1p5mKMwcQVCeQNVkzpUZqJ,3,2023-11-19T22:13:52Z
0,6luBKkFUt5wTwz7hpLhp12,1p5mKMwcQVCeQNVkzpUZqJ,4,2023-02-24T10:16:03Z
...,...,...,...,...
0,4Ouhoi2lAhrLJKFzUqEzwl,1p5mKMwcQVCeQNVkzpUZqJ,1721,2022-12-21T22:09:59Z
0,4H5oOXqZoxaHcoiJA4hrWk,1p5mKMwcQVCeQNVkzpUZqJ,1722,2022-12-21T22:09:59Z
0,6u5lHozM0oGj2l16lTF3KP,1p5mKMwcQVCeQNVkzpUZqJ,1723,2022-12-21T22:09:59Z
0,7oAuqs6akGnPU3Tb00ZmyM,1p5mKMwcQVCeQNVkzpUZqJ,1724,2022-12-21T22:09:59Z


In [10]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Power Down Playlist with Sian Eleri")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Power Down Playlist with Sian Eleri"
description = "Songs from the Power Down Playlist with Sian Eleri in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1354/1354 [01:01<00:00, 22.09it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,7dZ83x0X0QnZfm1mvJ83bn,3Tqe2k7TlEB5OatSMKeppu,0,2023-10-15T13:11:55Z
0,2AbAZwor5RySDTcrYdsVgb,3Tqe2k7TlEB5OatSMKeppu,1,2023-10-01T22:30:53Z
0,1pXfj151MyB7kWMkY4DYdt,3Tqe2k7TlEB5OatSMKeppu,2,2023-03-17T17:06:05Z
0,4jfgYiB2gQJ0mZSPcGJgIa,3Tqe2k7TlEB5OatSMKeppu,3,2022-12-21T22:10:44Z
0,3kxA3vI9OfWuwCyykd1EIo,3Tqe2k7TlEB5OatSMKeppu,4,2023-10-01T22:30:53Z
...,...,...,...,...
0,1TLHZcMQRMgW3N7hjh9Zlu,3Tqe2k7TlEB5OatSMKeppu,1349,2022-12-21T22:10:45Z
0,0tNuJpBgtE65diL6Q8Q7fI,3Tqe2k7TlEB5OatSMKeppu,1350,2022-12-21T22:10:45Z
0,3Z3wyUUPcKyw4nJkW2PqBA,3Tqe2k7TlEB5OatSMKeppu,1351,2022-12-21T22:10:45Z
0,32fdWZrgtAX2VRqCGkIuaF,3Tqe2k7TlEB5OatSMKeppu,1352,2023-10-01T22:30:54Z


In [11]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'BBC Introducing on Radio 1')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'BBC Introducing on Radio 1'
description = "Songs from BBC Introducing on Radio 1 in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

100%|██████████| 1246/1246 [00:00<00:00, 3572.33it/s]


,Track ID,Playlist ID,Position Number,Date Added
0,5jZ0Q2yRYnI961sJbyWlK0,07B5mpuLVdVpgSycsqn4Qc,0,2023-11-19T22:16:01Z
0,47e0ZFrxuIfoAvhLPdq5zy,07B5mpuLVdVpgSycsqn4Qc,1,2023-11-19T22:16:01Z
0,0nLRcAbhEuVtlJzMGS9elE,07B5mpuLVdVpgSycsqn4Qc,2,2023-11-19T22:16:01Z
0,1ZYKKqZx0ufiM5BK3RKrUI,07B5mpuLVdVpgSycsqn4Qc,3,2023-11-19T22:16:01Z
0,08FS0glTgPmSvyoNRqZZMx,07B5mpuLVdVpgSycsqn4Qc,4,2023-11-19T22:16:01Z
...,...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,1241,2022-12-21T22:11:18Z
0,5du3JWKm8DN37aOpeY0ir7,07B5mpuLVdVpgSycsqn4Qc,1242,2023-10-01T22:34:26Z
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,1243,2022-12-21T22:11:18Z
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,1244,2022-12-21T22:11:18Z


In [7]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Steve Lamacq")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Steve Lamacq"
description = "Songs from Steve Lamacq's Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks, add_to_end=True)

KeyboardInterrupt: 

In [12]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

In [12]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')